In [1]:
import os
import sys
import random
import numpy as np
import cv2

In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Activation, Dropout, UpSampling2D,Conv2D,MaxPooling2D,concatenate

In [3]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Activation,AveragePooling2D ,ReLU ,Add,Dense,Reshape,Multiply, Dropout, UpSampling2D,Conv2D,MaxPooling2D,concatenate,add,GlobalAveragePooling2D,GlobalMaxPool2D,Multiply

In [4]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from tensorflow.keras.models import load_model

In [5]:
train_images =np.load("D:/projects/Thesis/Dataset/D_img_512.npy").astype(np.float32)
labels = np.load("D:/projects/Thesis/Dataset/D_mask_512.npy").astype(np.float32)
train_images = np.array(train_images)
labels = np.array(labels)
labels = labels / 255
train_images, labels = shuffle(train_images, labels)
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.3)
batch_size =2
epochs = 200
pool_size = (2, 2)
input_shape = X_train.shape[1:]

In [6]:
import tensorflow.keras.backend as K

def dsc(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dsc(y_true, y_pred)



def IOU(y_true, y_pred):

    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    thresh = 0.5

    y_true = K.cast(K.greater_equal(y_true, thresh), 'float32')
    y_pred = K.cast(K.greater_equal(y_pred, thresh), 'float32')

    union = K.sum(K.maximum(y_true, y_pred)) + K.epsilon()
    intersection = K.sum(K.minimum(y_true, y_pred)) + K.epsilon()

    iou = intersection/union

    return iou

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16,VGG19,ResNet50

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, ZeroPadding2D
def conv_block1(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def conv_block2(input, num_filters):
    x = Conv2D(num_filters, 7, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x
def deconv(x, kernelsize,fi):
    #fi=x.shape[-1]
    fx = Conv2DTranspose(fi, kernelsize, activation='relu', padding='same')(x)
    fx = BatchNormalization()(fx)
    fx =Conv2DTranspose(fi, kernelsize, padding='same')(fx)
    out = BatchNormalization()(fx)
    return out

In [8]:
def CECA(x1,x2,r):
    
    
    f=x1.shape[-1]
    x=concatenate([x1,x2], axis = 3)
    a_pool = GlobalAveragePooling2D()(x)
    m_pool = GlobalMaxPool2D()(x) 
    Dense_layer1 = Dense (f/r , activation = 'relu') 
    Dense_layer2 = Dense (f, activation = 'relu') 
    avg_out = Dense_layer2 (Dense_layer1(a_pool)) 
    max_out = Dense_layer2 (Dense_layer1( m_pool))
    channel = add([ avg_out,max_out]) 
    channel = Activation('sigmoid') (channel) 
    channel = Reshape ((1,1,f)) (channel)  
    C_out = Multiply()([x1,x2,channel])
    return C_out
    
def GAM (il,ih):                     
    ###### Spatial Attention ########
    f=ih.shape[-1]
    il =Conv2D(1,(3,3),strides=1,padding ='same') (il)
    a_pool = GlobalAveragePooling2D()(ih)
    m_pool = GlobalMaxPool2D()(ih) 
    Dense_layer1 = Dense (f , activation = 'relu') 
    Dense_layer2 = Dense (f, activation = 'relu') 
    avg_out = Dense_layer2 (Dense_layer1(a_pool)) 
    max_out = Dense_layer2 (Dense_layer1( m_pool))
    channel = add([ avg_out,max_out]) 
    channel = Activation('sigmoid') (channel) 
    channel = Multiply()([il,channel])
    s=add([ channel,ih]) 
    
    av_pooling=AveragePooling2D(pool_size=(1,1))(s)
    ma_pooling=MaxPooling2D(pool_size=(1,1))(s)
    spatial=concatenate([av_pooling,ma_pooling], axis = 3)
    spatial =Conv2D(1,(7,7),strides=1,padding ='same') (spatial) 
    
    
    spatial= Activation('sigmoid')(spatial)
      
    S_out = Multiply()([s,spatial])
    
    return S_out

In [9]:
def AFM(i1,i2):
    
    x = Concatenate()([i1,i2])
    
    m_pool = MaxPooling2D(pool_size=(1,1))(x)
    
    w =Conv2D(1,(3,3),strides=1,padding ='same') (m_pool) 
    w= Activation('sigmoid')(w)
    I=Multiply()([i1,w])
    J= Concatenate()([i2, I])
    
    return J
    

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121

inputs = keras.layers.Input((512,512,3))


""" Encoder """
s1 = conv_block1(inputs,16)  
pool1 = MaxPooling2D(pool_size=(2, 2))(s1)
s2 = conv_block2(inputs,16) 
pool2 = MaxPooling2D(pool_size=(2, 2))(s2)
E1=CECA(pool1,pool2,8)


s3 = conv_block1(pool1,16) 
pool3 = MaxPooling2D(pool_size=(2, 2))(s3)
s4= conv_block2(pool2,16) 
pool4 = MaxPooling2D(pool_size=(2, 2))(s4)
E2=CECA(pool3,pool4,8)



s5 = conv_block1(pool3,32)  
pool5 = MaxPooling2D(pool_size=(2, 2))(s5)
s6= conv_block2(pool3,32) 
pool6 = MaxPooling2D(pool_size=(2, 2))(s6)
E3=CECA(pool5,pool6,8)


s7 = conv_block1(pool5,64) 
pool7 = MaxPooling2D(pool_size=(2, 2))(s7)
s8= conv_block2(pool6,64)  
pool8 = MaxPooling2D(pool_size=(2, 2))(s8)
E4=CECA(pool7,pool8,8)





s9 = conv_block1(pool7,128) 
pool9 = MaxPooling2D(pool_size=(2, 2))(s9)
s10= conv_block2(pool8,128)  
pool10 = MaxPooling2D(pool_size=(2, 2))(s10)
E5=CECA(pool9,pool10,8)







E5=UpSampling2D(size = (2,2))(E5)
D1=GAM(E5,E4)
D1=deconv(D1, 3,128)


D2=UpSampling2D(size = (2,2))(D1)
D2=GAM(D2,E3)
D2=deconv(D2, 3,64)


D3=UpSampling2D(size = (2,2))(D2)
D3=GAM(D3,E2)
D3=deconv(D3, 3,32)

D4=UpSampling2D(size = (2,2))(D3)
D4=GAM(D4,E1)
D4=deconv(D4, 3,16)

D5=UpSampling2D(size = (2,2))(D4)

D5=deconv(D5, 3,16)

############ Decoder #############

""" Outputs """
outputs = Conv2D(3, 1, padding="same", activation="sigmoid")(D5)

model = Model(inputs, outputs)
    

datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,  loss = dice_loss, metrics = ['dsc','accuracy', 'IOU','precision_m', 'recall_m', 'f1_m'])
model.summary()

###### Saving the model ############
model.save('D:/projects/Thesis/Thesis_final_7.h5')
 
# load model
model = load_model('D:/projects/Thesis/Thesis_final_7.h5',custom_objects={'dice_loss':dice_loss,'IOU':IOU,'dsc':dsc,'precision_m':precision_m, 'recall_m':recall_m, 'f1_m':f1_m})
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
epochs=200, verbose=1, validation_data=(X_val, y_val))

model.save('D:/projects/Thesis/Thesis_final_7.h5')